In [21]:
import os
import boto3
import pandas as pd
from io import StringIO

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline

import mlflow
import mlflow.sklearn

In [2]:
os.environ['MLFLOW_TRACKING_URI'] = 'http://localhost:5050'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://localhost:9000'
os.environ['AWS_ACCESS_KEY_ID'] = 'Danil123'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'Danil1231'

In [3]:
# Инициализация клиента
s3 = boto3.client('s3',
                  endpoint_url='http://localhost:9000',
                  aws_access_key_id='Danil123',
                  aws_secret_access_key='Danil123')

In [4]:
obj = s3.get_object(Bucket='datasets', Key='kinopoisk_train.csv')
data = obj['Body'].read().decode('utf-8')
df = pd.read_csv(StringIO(data))

In [5]:
df

,review,sentiment
0,"Смотрел фильм несколько раз, с первого раза вы...",neu
1,Я даже не знаю с чего начать... Сериал этот я ...,neg
2,Одна вещь порождает следующую. Человек имеет с...,pos
3,"«Журнал 64», как и вся серия адаптации книг эт...",neg
4,"Однажды утром просматривая почту, Дон Джонстон...",pos
...,...,...
19746,Перед нами одна из многочисленных короткометра...,pos
19747,Очень редко смотрю современные комедии - как н...,pos
19748,При просмотре третьей части меня никак не поки...,neg
19749,Как всегда поражаюсь удивительной сдержанности...,pos


In [6]:
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'], test_size=0.2, random_state=42)

In [7]:
X_train

5770     -Я была дурой...\n-Мы все были полными дуракам...
6202     «Малавита» - криминальная черная детективная к...
10373    “Кристофер Робин” - трогательная история о пов...
2556     Отличная  идея режиссера – разделить фильм на ...
380      Вера, любовь, Бог. Вопросы важные в жизни кажд...
                               ...                        
11284    Фильм однозначно не понравился. Во многих стар...
11964    На самом деле, чем лучше книга, тем труднее её...
5390     После того, как в конце 50-х, обогнав Бурвиля ...
860      Гравити Фолз - мультсериал студии Disney, расс...
15795    Тихоокеанский рубеж прошёл мимо меня, и при пр...
Name: review, Length: 15800, dtype: object

In [8]:
# Векторизация
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [9]:
# Обучение модели
clf = LogisticRegression()
clf.fit(X_train_vec, y_train)

c:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [34]:
pipelines = {}

pipelines['Logreg'] = Pipeline([
    ('vect', CountVectorizer()),
    ('model', LogisticRegression()),
])

pipelines['SGD'] = Pipeline([
    ('vect', CountVectorizer()),
    ('model', SGDClassifier())
])

pipelines['Ridge'] = Pipeline([
    ('vect', CountVectorizer()),
    ('model', RidgeClassifier())
])


In [35]:
pipelines

{'Logreg': Pipeline(steps=[('vect', CountVectorizer()), ('model', LogisticRegression())]),
 'SGD': Pipeline(steps=[('vect', CountVectorizer()), ('model', SGDClassifier())]),
 'Ridge': Pipeline(steps=[('vect', CountVectorizer()), ('model', RidgeClassifier())])}

In [70]:
pipelines = {}

pipelines['Logreg'] = Pipeline([
    ('vect', CountVectorizer()),
])

pipelines['SGD'] = Pipeline([
    ('vect', CountVectorizer()),
])

pipelines['Ridge'] = Pipeline([
    ('vect', CountVectorizer()),
])

In [74]:
pipelines['Logreg'].fit_transform(X_train)

<15800x236893 sparse matrix of type '<class 'numpy.int64'>'
	with 3656129 stored elements in Compressed Sparse Row format>